In [ ]:
# 
# https://scikit-image.org/docs/dev/api/skimage.measure.html
#
from skimage.measure import LineModelND, ransac, CircleModel
from skimage import io, feature, color, measure, draw, img_as_float
from sklearn.metrics import mean_squared_error, r2_score

###############################################################
columns = ["iTr", "cluster_lable", "pixels", "photons", "ph_pixels", "x0start", "y0start", 
          "x0end", "y0end", "length", "mse", "r2"]

cfile = cy.open_(runI[0], tag=tag, posix=posix, verbose=False)
if cfile.x_resolution == 2048:
    rescale = 512 # orca flash
else:
    rescale = 576 # fusion


if not doRescale: 
    rescale = cfile.x_resolution
    
        
tscale = int(cfile.x_resolution/rescale)
print("rascale: ", tscale)
fiducial_cut = np.int(fiducial_cut/tscale)
#########################

m_image, s_image = cy.ped_(run_ped, path=mybasepath+'ped/')

#########################

th_image   = np.round(m_image + nsigma*s_image)
print ("light over Th: %.2f " % (th_image.sum()-m_image.sum()))
    
for nRi in range(len(runI)):
    try:
        # open file
        cfile = cy.open_(runI[nRi], tag=tag, posix=posix, verbose=True)
    except:
        print ('Problem in open file: ', runI[nRi])
        break
    #
    #
    # crea un db vuoto da riempire 
    df = pd.DataFrame(columns = columns)
    
    if max_pic>=cfile.max_pic or max_pic == -1:
        max_pic = cfile.max_pic
    # crea nome file di output
    file_out = (mybasepath+"out/cosmics_run%d_cmin_%d_cmax_%d_rescale_%d_nsigma_%.1f_ev_%d_ped_%d_ms_%d_rt_%d_%s.plk.gz" % 
                (runI[nRi], cimin, cimax, rescale, nsigma, max_pic, run_ped, min_samples, 
                 residual_threshold, version))
    #
    for iTr in range(0, max_pic): # max_image per tutte le imagini cfile.max_image
        #if iTr % 10 == 0: 
          # running & debug ...
        start_time = time.time()
        print ('Processing RUN: ', runI[nRi], 'Event: ', iTr)
        
          # end 

        image = rtnp.hist2array(cfile.file.Get(cfile.pic[iTr])).T
        raw_image       = np.subtract(image,m_image)
        rebin_image     = cy.rebin(raw_image, (rescale, rescale))  
        rebin_th_image  = cy.rebin((th_image-m_image), (rescale, rescale))
        
        for ic in range(max_tarcks):
            if ic>0:
                for i in range(xc.shape[0]):
                    edges[int(yc[i]), int(xc[i])]=0
            else:
                edges = (rebin_image > rebin_th_image) & (rebin_image < cimax)
                
            edges_fiducial = edges
            edges_fiducial[0:fiducial_cut,:]=0
            edges_fiducial[edges.shape[0]-fiducial_cut:edges.shape[0],:]=0
            edges_fiducial[:,0:fiducial_cut]=0
            edges_fiducial[:,edges.shape[1]-fiducial_cut:edges.shape[1]]=0
            
            points          = np.array(np.nonzero(edges_fiducial)).T.astype(float)
            
          
            #
            # robustly fit line only using inlier data with RANSAC algorithm
            # min_samples quanti punti prende vicino per il fit (2)
            # residual_threshold quanti punti accumola attorno al fit (1)
            # random_state per dargli sempre lo stesso seme inziale
            #
            # print (min_samples,residual_threshold, max_trials, random_state)
            model_robust, inliers = ransac(points, LineModelND, min_samples=min_samples,
                                           residual_threshold=residual_threshold, max_trials=max_trials, 
                                           random_state=random_state)
            outliers = inliers == False

            # generate coordinates of estimated models
            line_x = np.arange(0, rebin_image.shape[1])
            # se vuoi pure il fit linere semplice 
            # model = LineModelND()
            # model.estimate(points)
            # line_y = model.predict_y(line_x)
            line_y_robust = model_robust.predict_y(line_x)

            xc = points[inliers, 1]
            yc = points[inliers, 0]        
        
            for j in range(0, xc.shape[0]):
                x=int(xc[j])
                y=int(yc[j])
                if j == 0:
                    x0start = x*tscale
                    y0start = y*tscale
            x0end   = x*tscale
            y0end   = y*tscale
            
            length  = np.sqrt((x0end - x0start)**2 + (y0end - y0start)**2)
            ph, dim = cy.cluster_par(xc, yc, rebin_image)
            ph      = ph*tscale*tscale
            dim     = dim*tscale*tscale
            
            mse = mean_squared_error(points[inliers][:,1], model_robust.predict_y(points[inliers][:,0]))
            r2  = r2_score(points[inliers][:,1], model_robust.predict_y(points[inliers][:,0]))
#             if dim<2000 or mse>5 or r2<0.95:
#                 print ("track: ", ic)
#                 break
            df  = df.append({columns[0]:iTr, columns[1]:ic, columns[2]:dim, columns[3]:ph, columns[4]:ph/dim, 
                columns[5]:x0start, columns[6]:y0start, columns[7]:x0end, columns[8]:y0end, 
                columns[9]:length, columns[10]:mse, columns[11]:r2},
                ignore_index=True)
            if (iTr % 10 == 0 and ic == 0) or debug:
                print ("DEBUG: number of points, clusters: " +str(dim), ic) 
                print ([str(columns[i])+': {:.2f}'.format(x) for i, x in enumerate (df.tail(1).values[0])])
                # print(model.params)
                print(model_robust.params)

                fig, ax = plt.subplots (1,2, figsize=(20,10))
                ax[0].imshow(rebin_image, cmap='jet', vmin=0, vmax=20)
                ax[1].imshow(edges_fiducial, cmap='YlGnBu', vmin=0,vmax=1)
                ax[1].plot(xc, yc, '.g', alpha=0.6, label='data', markersize=6)
                ax[1].plot(line_y_robust, line_x, '--r', label='mse: {:.2f}\n r2: {:.4f}'.format(mse, r2))
                ax[1].set_xlim(0,rebin_image.shape[1]-1)
                ax[1].set_ylim(rebin_image.shape[0]-1,0)
                ax[1].axvline(fiducial_cut, color='k')
                ax[1].axvline(edges.shape[1]-fiducial_cut, color='k')
                ax[1].axhline(fiducial_cut, color='k')
                ax[1].axhline(edges.shape[0]-fiducial_cut, color='k')
                print("ph {:.0f}, dim: {:d}".format(ph, dim))

                print("x,y start: {:d}, {:d}, x,y stop: {:d}, {:d}, length: {:.0f}: ".format(
                    x0start, y0start, x0end, y0end, length))
                print('Mean squared error: {:.2f}'.format(mse))
                # The coefficient of determination: 1 is perfect prediction
                print('Coefficient of determination: {:.4f}'.format(r2))
                ax[1].legend()
                plt.show()

        print ("Elapsed time 10 events: {:.1f}".format(time.time() - start_time))


        start_time = time.time()
    df.to_pickle(file_out, compression='gzip')
    print ("out file", file_out)